Tutorial on U-NET: https://www.youtube.com/watch?v=IHq1t7NxS8k

In [80]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
from torchvision import transforms as T

In [81]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    
    def forward(self, x):
        return self.conv(x)


class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
        
    
    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)   
    
def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    assert preds.shape == x.shape
    

if __name__ == "__main__":
    test()

In [82]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

In [83]:
class CamVidDataSet(Dataset):
    def __init__(self, imgs_path, labels_path, transform=None):
        self.imgs_path = imgs_path
        self.labels_path = labels_path #mask
        self.transform = transform
        
        self.imgs = os.listdir(self.imgs_path)
        self.labels = list(map(lambda x: x[:-4] + '_L.png', self.imgs)) #see train_labels
        
    def __len__(self):
        return len(self.imgs)
        
    def __getitem__(self, idx):
        img_loc = os.path.join(self.imgs_path, self.imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        
        label_loc = os.path.join(self.labels_path, self.labels[idx])
        label = Image.open(label_loc).convert("RGB")
        
        if self.transform is not None:
            image = self.transform(image)
            label = self.transform(label)
        
        image_tensor = T.Compose([T.ToTensor()])(image) 
        label_tensor = T.Compose([T.PILToTensor()])(label)
        
        return image_tensor, label_tensor

In [84]:
from torch.utils.data import DataLoader

In [87]:
transformation = T.Compose([T.Resize(input_size, T.InterpolationMode.BICUBIC)])
train_data = CamVidDataSet(
        imgs_path="./CamVid/train/",
        labels_path="./CamVid/train_labels/",
        transform=transformation,
    )

In [73]:
def get_loaders(
    train_dir,
    train_labeldir,
    val_dir,
    val_labeldir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_data = CamVidDataSet(
        imgs_path=train_dir,
        labels_path=train_labeldir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_data = CamVidDataSet(
        imgs_path=val_dir,
        labels_path=val_labeldir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

In [74]:
input_size = (128, 128)
transformation = T.Compose([T.Resize(input_size, T.InterpolationMode.BICUBIC)])

data_loader, data_loader_test = get_loaders(
        train_dir="./CamVid/train/",
        train_labeldir="./CamVid/train_labels/",
        val_dir="./CamVid/val/",
        val_labeldir="./CamVid/val_labels/",
        batch_size=3,
        train_transform=transformation,
        val_transform=transformation,
        num_workers=4,
        pin_memory=True
    )

In [75]:
#input_size = (128, 128)
#transformation = transforms.Compose([T.Resize(input_size, T.InterpolationMode.BICUBIC)])
#camvid_train = CamVidDataSet("./CamVid/train/", "./CamVid/train_labels/", transform=transformation)
#camvid_test = CamVidDataSet("./CamVid/val/", "./CamVid/val_labels/", transform=transformation)

# define training and validation data loaders
#data_loader = torch.utils.data.DataLoader(
#    camvid_train, batch_size=2, shuffle=True, num_workers=4)

#data_loader_test = torch.utils.data.DataLoader(
#    camvid_test, batch_size=1, shuffle=False, num_workers=4)

In [76]:
#from torch.optim.lr_scheduler import StepLR

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = UNET(in_channels=3, out_channels=3)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]

In [77]:
def train_one_epoch(data_loader, model, optimizer, loss_fn, scaler):
    for batch_idx, (data, targets) in enumerate(data_loader):
        data = data.to(device)
        targets = targets.float().to(device)
        
        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            print(predictions)
            print(predictions.shape)
            loss = loss_fn(predictions, targets)
            
        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        #loop.set_postfix(loss=loss.item())

In [78]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params, lr=0.01)
scaler = torch.cuda.amp.GradScaler()

num_epochs = 3

for epoch in range(num_epochs):
    train_one_epoch(data_loader, model, optimizer, loss_fn, scaler)

tensor([[[[ 0.2443,  0.4067,  0.8369,  ...,  0.5034, -0.4668, -0.6250],
          [ 0.4539,  0.0213, -0.2108,  ..., -0.6089, -0.7017, -0.2235],
          [-0.0787, -0.3960,  0.8530,  ...,  0.1588, -0.5825,  0.0978],
          ...,
          [-0.0436, -0.0547,  0.4263,  ..., -0.1217, -0.3823,  0.5098],
          [-0.2230,  0.2607, -0.0968,  ...,  0.1407, -0.2086,  0.5693],
          [-0.4233, -0.2372, -0.2905,  ...,  0.0609, -0.0867,  0.5571]],

         [[-0.1826,  0.9595,  0.7344,  ...,  1.1367,  1.9434,  1.0723],
          [ 0.6626,  0.0330,  0.0114,  ...,  0.6675,  1.1035,  0.0062],
          [-0.3052, -0.3740,  0.1152,  ..., -0.5967,  1.2021,  0.2462],
          ...,
          [ 0.4229, -0.3169,  0.1106,  ..., -0.0079,  0.3704,  0.2727],
          [ 0.3635, -0.6216,  0.0223,  ...,  0.0647,  0.2815,  0.2578],
          [ 0.5293,  0.1503,  0.3633,  ...,  0.5518,  0.6025,  0.5771]],

         [[-0.4070, -0.8076, -0.6611,  ..., -1.6611, -1.7100,  0.0323],
          [-0.6299,  0.0986,  

tensor([[[[ 8.1055e-02, -1.2976e-01,  2.9907e-01,  ...,  1.2744e-01,
            1.1377e-01,  2.1228e-01],
          [ 1.3782e-01,  3.4692e-01,  2.7856e-01,  ..., -7.7734e-01,
           -4.9854e-01,  3.4863e-01],
          [ 3.9978e-03,  2.8369e-01,  2.7515e-01,  ...,  4.0698e-01,
            8.3862e-02,  7.0361e-01],
          ...,
          [-1.1597e-02,  3.0869e-02,  2.3657e-01,  ..., -1.6455e-01,
           -4.4556e-01,  3.7915e-01],
          [ 2.2559e-01,  1.4661e-01,  2.2125e-02,  ...,  2.6123e-01,
           -1.2024e-02,  6.5527e-01],
          [ 3.2422e-01,  7.3853e-02,  1.4246e-01,  ...,  4.6997e-02,
           -6.8237e-02,  3.5156e-01]],

         [[ 3.7598e-01,  6.7041e-01,  1.1902e-01,  ...,  8.7451e-01,
            4.2114e-01,  6.0205e-01],
          [ 1.2903e-01, -2.6489e-01,  2.3560e-02,  ...,  1.6162e-01,
            4.5135e-02, -1.2927e-01],
          [ 3.2959e-01, -8.2153e-02,  9.1736e-02,  ..., -1.1316e-01,
           -6.0181e-02,  2.1765e-01],
          ...,
     

tensor([[[[-3.6230e-01, -4.1968e-01, -3.7256e-01,  ..., -1.5833e-01,
           -1.4636e-01, -1.1755e-01],
          [-2.6392e-01, -3.3911e-01, -1.7224e-01,  ...,  2.3547e-01,
            6.2134e-02, -9.6191e-02],
          [-2.7197e-01, -2.7588e-01, -8.2886e-02,  ...,  4.5776e-01,
            1.8384e-01,  4.9194e-02],
          ...,
          [-8.5205e-02, -1.2317e-01, -8.8013e-02,  ..., -4.2816e-02,
           -1.6296e-01, -1.1365e-01],
          [-1.8359e-01, -1.7126e-01, -7.8247e-02,  ..., -8.3862e-02,
           -1.4209e-01, -8.5815e-02],
          [-1.2769e-01, -1.1243e-01, -9.6436e-02,  ..., -6.2988e-02,
           -7.8125e-02, -4.9133e-03]],

         [[-1.9482e-01, -3.2764e-01, -3.2129e-01,  ...,  8.9355e-01,
            6.4844e-01,  3.7939e-01],
          [-4.4653e-01, -8.4766e-01, -6.7188e-01,  ...,  1.1338e+00,
            1.0918e+00,  6.7578e-01],
          [-3.6865e-01, -5.9180e-01, -6.3770e-01,  ...,  1.1094e+00,
            1.5010e+00,  1.1016e+00],
          ...,
     

tensor([[[[0.2690, 0.4990, 0.8164,  ..., 0.8770, 0.5488, 0.2424],
          [0.5000, 1.4307, 2.0703,  ..., 2.0742, 1.3643, 0.4714],
          [0.7939, 1.9473, 2.7930,  ..., 2.4062, 1.5527, 0.5068],
          ...,
          [0.3857, 0.4146, 0.4614,  ..., 0.3782, 0.3936, 0.3838],
          [0.3687, 0.3953, 0.4265,  ..., 0.3606, 0.3611, 0.3723],
          [0.3838, 0.3843, 0.4165,  ..., 0.3816, 0.3926, 0.3901]],

         [[1.2588, 1.9561, 2.6074,  ..., 2.7402, 2.1309, 1.3213],
          [1.7549, 3.3730, 4.4922,  ..., 4.4961, 3.2871, 1.7324],
          [2.1758, 4.1836, 5.5742,  ..., 4.9688, 3.4688, 1.6895],
          ...,
          [0.4570, 0.3606, 0.3599,  ..., 0.3545, 0.3254, 0.4648],
          [0.4414, 0.3306, 0.3423,  ..., 0.3120, 0.3079, 0.4395],
          [0.5312, 0.4268, 0.4287,  ..., 0.4404, 0.4360, 0.5439]],

         [[1.1025, 1.0879, 1.2461,  ..., 1.2715, 1.1611, 1.0781],
          [1.0117, 1.3721, 1.5674,  ..., 1.5742, 1.3799, 1.0566],
          [1.0977, 1.4893, 1.7266,  ..., 1

tensor([[[[0.6973, 0.5723, 0.5308,  ..., 0.4949, 0.5103, 0.6436],
          [0.6377, 0.5078, 0.6934,  ..., 0.6240, 0.4939, 0.5459],
          [0.5840, 0.6211, 1.1660,  ..., 1.0732, 0.6338, 0.4827],
          ...,
          [1.0400, 1.2832, 1.3223,  ..., 1.4229, 1.4336, 1.0664],
          [1.0234, 1.2520, 1.2939,  ..., 1.4141, 1.3945, 1.0430],
          [0.9492, 1.0400, 1.0752,  ..., 1.0381, 1.0254, 0.9258]],

         [[0.7656, 0.6650, 0.8408,  ..., 0.7803, 0.6504, 0.7031],
          [0.6846, 0.6904, 1.2324,  ..., 1.1592, 0.6836, 0.6055],
          [0.7314, 1.0469, 1.9746,  ..., 1.8242, 1.0820, 0.6074],
          ...,
          [0.9102, 0.9629, 0.9902,  ..., 0.9395, 0.8896, 0.8799],
          [0.8857, 0.9229, 0.9541,  ..., 0.9053, 0.8662, 0.8467],
          [0.8984, 0.8838, 0.9023,  ..., 0.8604, 0.8311, 0.8350]],

         [[1.1943, 0.9385, 0.9062,  ..., 0.8218, 0.8613, 1.0488],
          [1.0303, 0.7427, 0.8428,  ..., 0.7529, 0.6548, 0.8594],
          [0.9873, 0.7744, 1.1299,  ..., 1

...

tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]],


        [[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan

tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]],


        [[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan

tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]],


        [[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan

tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]],


        [[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan

tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]],


        [[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan